# ONTOP-Dremio Playground


**NOTE**: this notebook is written in JavaScript, so you need Jupyter + a JavaScript interpreter to run it. You may use the Deno interpreter, which is powerful and comes with a prebuild support for notebooks
```bash
curl -fsSL https://deno.land/install.sh | sh  # Install Deno
"$HOME/.deno/bin/deno" jupyter --install      # Add Deno to Jupyter
```
now just restart VSCode/Jupyter and you should see Deno amongst the kernels

In [1]:
const hello = "hello world!"
console.log(hello)

hello world!


## Setup

In [85]:
// Connect to postgres
import { Client } from "https://deno.land/x/postgres/mod.ts";

const createClient = async (port: number) : Promise<Client> => {
  const client = new Client({
    user: "user",
    database: "example",
    password: "pass",
    hostname: "localhost",
    port,
  });
  await client.connect();
  return client
}

const loadSQLData = async (client: Client, filePath : string, logSql = false) => {
  const sql = await Deno.readTextFile(filePath);
  await client.queryArray(sql);
  if(logSql) console.log(sql)
  //console.log("SQL file loaded successfully!");
}

const clearDatabases = (...args: Client[]) => Promise.all(args.map(client => loadSQLData(client, './data/postgres/clean.sql')))

const logTables = async (client: Client) => {
  const result = await pg1.queryObject(` SELECT table_name FROM information_schema.tables
    WHERE table_schema = 'public'   AND table_type = 'BASE TABLE';`);
  console.log("Tables:", result.rows);
}

const pg1 = await createClient(5531)
const pg2 = await createClient(5532)
const pg3 = await createClient(5533)



In [52]:
// Connect to dremio and add postgres sources if not already there
const hostname = "http://localhost:9047";
const endpoint = `${hostname}/api/v3`;
const userName = 'dremioUser'
const password = 'dremioPass1' // password needs a number, otherwise you will get error 403

const post = async (url: string, data: Record<string, unknown>) => {
    const response = await fetch(url, {
        method: "POST",
        headers: {
        "Content-Type": "application/json",
        },
        body: JSON.stringify(data),
    });
    if (!response.ok) throw new Error(`HTTP error! status: ${response.status}`);  
    return response.json();
}

// create first user if none
const firstName = 'admin'
const lastName = ''
const email = 'luca.fabbian.1999@gmail.com'
const createdAt = Date.now()
const response = await fetch(`${hostname}/apiv2/bootstrap/firstuser`, {
    method: "PUT",
    headers: {
      "Authorization": '_dremionull',
      "Content-Type": "application/json",
    },
    body: JSON.stringify({userName, password, firstName, lastName, email, createdAt}),
});

if (!response.ok ){
  if(response.status !== 400) throw new Error(`Unexpected HTTP error! status: ${response.status}`);
  console.log("User alredy there, no need to create a new one")
}else{
  console.log("New user created", await response.json())
}


// get auth token
const {token, expires } = await post(`${hostname}/apiv2/login`, { userName, password })
console.log('Auth token:', token, '\nExpires at:', (new Date(expires)).toLocaleTimeString('en-GB'), '(30hours)')

// helper function to make dremio api calls
const dremio = async(method: 'GET'|'POST', url: string, data: Record<string, unknown>)  => {
    const response = await fetch(`${endpoint}${url}`, {
        method,
        headers: {
            'Authorization': `Bearer ${token}`,
            "Content-Type": "application/json",
        },
        body: JSON.stringify(data),
    });

    if (!response.ok){
      const e = new Error(`HTTP error! status: ${response.status}`)
      e.httpStatus = response.status
      throw e
    }
    return response.json();
}


const setupPostgres = async (name: string, hostname = "postgres", port = "5432") => await dremio('POST', '/catalog', {
  entityType: 'source',
  type: "POSTGRES",
  name,
  config: {
      hostname,
      port,
      databaseName: "example",
      username: "user",
      password: "pass",
      authenticationType: "MASTER",
      fetchSize: 200,
      useSsl: false,
      encryptionValidationMode: "CERTIFICATE_AND_HOSTNAME_VALIDATION",
      maxIdleConns: 8,
      idleTimeSec: 60,
      queryTimeoutSec: 0
  },
})

try {
  await Promise.all([
    setupPostgres("postgres1", "postgres1"),
    setupPostgres("postgres2", "postgres2"),
    setupPostgres("postgres3", "postgres3"),
  ]);
}catch(e){
  if(e.httpStatus === 409) console.log("Postgres sources already there - status 409")
  else throw e
}


User alredy there, no need to create a new one
Auth token: fovfn9636qk2520fpu9ur8r1hc 
Expires at: 19:10:09 (30hours)
Postgres sources already there - status 409


In [83]:
// Perform an ontop query
// ontopQuery.ts
import * as uuid from "jsr:@std/uuid";


async function runOntopQuery(options: {
    facts?: string,
    constraint?: string,
    dbMetadata?: string,
    dbDriver?: string,
    dbPassword?: string,
    dbUrl?: string,
    enableAnnotations?: boolean,
    factsBaseIri?: string,
    factsFormat?: string,
    lenses?: string,
    mapping: string,   // Required
    output?: string,
    properties?: string,
    query: string,     // Required
    sparqlRules?: string,
    ontology?: string,
    dbUser?: string,
    xmlCatalog?: string
  }) {
    const cmd = ["docker", "compose", "exec", "-u", "root",  "ontop", "ontop", "query"];
  
    if (options.facts) cmd.push("-a", options.facts);
    if (options.constraint) cmd.push("-c", options.constraint);
    if (options.dbMetadata) cmd.push("-d", options.dbMetadata);
    if (options.dbDriver) cmd.push("--db-driver", options.dbDriver);
    if (options.dbPassword) cmd.push("--db-password", options.dbPassword);
    if (options.dbUrl) cmd.push("--db-url", options.dbUrl);
    if (options.enableAnnotations) cmd.push("--enable-annotations");
    if (options.factsBaseIri) cmd.push("--facts-base-iri", options.factsBaseIri);
    if (options.factsFormat) cmd.push("--facts-format", options.factsFormat);
    if (options.lenses) cmd.push("-l", options.lenses);
    cmd.push("-m", options.mapping);  // Required field
    if (options.output) cmd.push("-o", options.output);
    if (options.properties) cmd.push("-p", options.properties);
    cmd.push("-q", options.query);  // Required field
    if (options.sparqlRules) cmd.push("--sparql-rules", options.sparqlRules);
    if (options.ontology) cmd.push("-t", options.ontology);
    if (options.dbUser) cmd.push("-u", options.dbUser);
    if (options.xmlCatalog) cmd.push("-x", options.xmlCatalog);
  
    // Run the command
    const process = Deno.run({
      cmd: cmd,
      stdout: "piped",
      stderr: "piped"
    });
  
    const output = await process.output();
    const error = await process.stderrOutput();
  
    const decoder = new TextDecoder();
  

    return  decoder.decode(output)
    if (error.length > 0) {
      console.error("Error:", decoder.decode(error));
      console.log("Output:", decoder.decode(output));

      return decoder.decode(error);
    } else {
      console.log("Output:", decoder.decode(output));
      return decoder.decode(output);
    }
  }

  const runQuery = async (args: Record<string, string> = {}, queryString : string = undefined) => {

    const outFile = uuid.v1.generate() + '-gitexclude.csv'
    const queryFile = uuid.v1.generate() + '-gitexclude.sparql'

    if(queryString) await Deno.writeTextFile(`./data/ontology/${queryFile}`, queryString)

    const res = await runOntopQuery({
      mapping: "/opt/ontop/input/1/mapping.ttl",
      query: `/opt/ontop/input/${queryFile}`,
      dbUrl: "jdbc:dremio:direct=dremio:31010",
      dbUser: "dremioUser",
      dbPassword: "dremioPass1",
      output: `/opt/ontop/input/${outFile}`,
      ...args
    });

    console.log(res)
    const text = await Deno.readTextFile(`./data/ontology/${outFile}`);
    await Deno.remove(`./data/ontology/${outFile}`);
    if(queryString) await Deno.remove(`./data/ontology/${queryFile}`)

    return text
  }


sub,pred,obj
http://example.org/patients/1,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://example.org/Patient
http://example.org/patients/2,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://example.org/Patient



In [ ]:
await logTables(pg1)

In [ ]:
// inspect source(s)
await dremio('GET', '/catalog')

In [86]:
await clearDatabases(pg1, pg2, pg3)




await loadSQLData(pg1, './data/postgres/hospital.sql')
await loadSQLData(pg2, './data/postgres/hospital.sql')

In [88]:
console.log(await runQuery({}, `
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    SELECT * WHERE {
      ?sub ?pred ?obj .
    } 
    LIMIT 2
    
    `))
    

NotFound: No such file or directory (os error 2): readfile './data/ontology/4cbde1f0-7111-11ef-941e-fd1d69fe9a1e-gitexclude.csv'